In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('binance_data.csv',header=None)
columns = ['timestamp','open','high','low','close','volume']
data.columns = columns
data["timestamp"] = pd.to_datetime(data["timestamp"])
#data=data.drop(['timestamp'],axis = 1 )
data

,timestamp,open,high,low,close,volume
0,2024-11-25 18:23:00,97930.13,97982.54,97928.20,97982.53,18.27246
1,2024-11-25 18:24:00,97982.54,97999.98,97944.00,97949.99,9.10211
2,2024-11-25 18:25:00,97949.99,97984.25,97949.99,97984.24,10.24976
3,2024-11-25 18:26:00,97984.24,97984.25,97942.00,97942.00,9.39547
4,2024-11-25 18:27:00,97942.01,97967.99,97892.06,97892.06,21.62757
...,...,...,...,...,...,...
414,2024-11-26 01:19:00,94800.00,94872.02,94768.00,94808.02,36.25977
415,2024-11-26 01:20:00,94808.02,94866.89,94736.00,94816.98,56.18272
416,2024-11-26 01:21:00,94816.98,94914.61,94800.00,94914.61,50.36162
417,2024-11-26 01:22:00,94914.61,94971.83,94824.00,94863.49,42.76734


In [3]:
X_raw = data[['open', 'high', 'low', 'volume']].values
y_raw = data[['close']].values

In [4]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler() 

X = scaler_X.fit_transform(X_raw)
y = scaler_y.fit_transform(y_raw.reshape(-1, 1))

In [5]:
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

In [6]:
seq_length = 10  # Use 10 time steps for each sequence
X_seq, y_seq = create_sequences(X, y, seq_length)

In [7]:
X_seq[1]

array([[0.99951413, 1.        , 0.99831267, 0.00701744],
       [0.99021107, 0.99532548, 1.        , 0.0100518 ],
       [1.        , 0.99532548, 0.99774929, 0.00779308],
       [0.98793032, 0.99049345, 0.98368164, 0.0401344 ],
       [0.9736542 , 0.98118304, 0.98368164, 0.01787383],
       [0.97592638, 0.97911472, 0.97465063, 0.02504029],
       [0.96651471, 0.966369  , 0.97126471, 0.05423671],
       [0.96798091, 0.96999153, 0.9733295 , 0.00669015],
       [0.97563771, 0.9768265 , 0.97958022, 0.03614002],
       [0.97418295, 0.96847892, 0.94738858, 0.1195335 ]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=10)

In [9]:
n_features = X_train.shape[2]  # Number of predictors
n_hidden = 7  # Number of hidden units
n_output = 1  # Single output (price)
learning_rate = 0.001
epochs = 500

In [10]:
np.random.seed(10)
i_weight = np.random.rand(n_features, n_hidden) / 5 - 0.1    
h_weight = np.random.rand(n_hidden, n_hidden) / 5 - 0.1      
h_bias = np.random.rand(1, n_hidden) / 5 - 0.1               
o_weight = np.random.rand(n_hidden, n_output) * 50           
o_bias = np.random.rand(1, n_output)                         

outputs = np.zeros(len(X_train))
hiddens = np.zeros((len(X_train), n_hidden))
prev_hidden = None

In [11]:
def tanh(z):
    return np.tanh(z)
    
def tanh_derivative(z):
    return 1 - np.tanh(z) ** 2

In [12]:
for i in range(len(X_train)):  
    sequence = X_train[i]  
    
    for t in range(seq_length):  
        x = sequence[t].reshape(1, n_features)  

        # Multiply input by input weight
        xi = x @ i_weight

        if prev_hidden is not None:
            # Add previous hidden state 
            xh = xi + prev_hidden @ h_weight + h_bias
        else:
            xh = xi + h_bias

        # Apply activation function (tanh)
        xh = np.tanh(xh)
        prev_hidden = xh

    # Store the final hidden state for this sequence
    hiddens[i, :] = prev_hidden

    # Multiply by the output weight to compute the prediction
    xo = prev_hidden @ o_weight + o_bias
    outputs[i] = xo.flatten()  # Store the prediction

print("Outputs (scaled):", outputs)

Outputs (scaled): [3.06632028 4.76804742 4.83640609 1.64701584 1.17511355 2.4530729
 1.7633204  2.6625013  2.05642985 1.32514179 1.78866205 1.72902154
 2.89840512 1.15945209 1.68652814 4.85400889 2.67550162 1.86497614
 4.53467052 4.58781367 2.75940265 4.70918735 1.42568115 4.34985402
 4.69887607 4.24556285 2.58086607 1.75737902 1.72947551 3.11931736
 2.90235054 4.05145214 2.23058787 3.28155906 4.70136589 2.87977002
 1.55920123 4.88381092 0.91123681 4.46687586 2.31821815 2.89643382
 4.70226048 2.55236413 2.01834362 4.58330969 1.75513214 2.9592324
 1.77927035 1.62330177 4.45591862 1.90159584 1.50381194 1.86999356
 1.45429818 3.16165717 1.76251719 1.72555278 1.77540607 2.95274723
 4.85941368 1.75367064 2.82681241 2.44085862 1.10725078 1.75204557
 2.0935204  2.23595169 2.34723985 1.4449842  4.82406336 1.98690077
 4.46909678 1.7196423  2.86493511 1.84175118 2.6601755  4.6883316
 2.80758931 2.28448451 2.36965451 1.7966621  1.92416143 1.88089452
 3.12083993 2.37432553 1.28253959 4.73665754 4.

In [13]:
(outputs[0])

3.0663202792784037

In [14]:
hiddens[1]

array([ 0.22513637, -0.03467122, -0.00106172,  0.06112834,  0.17433056,
       -0.0597146 , -0.11630712])